In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [16]:


NETFLIX = 'netflix_titles.csv'
RATING_MAP = {
    'A': 'R',
    'UR': 'NR',
    'TV-Y7-FV': 'TV-Y7',
}
USECOLS = ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']

df = pd.read_csv(filepath_or_buffer=NETFLIX, encoding='iso8859_10', usecols=USECOLS, index_col=['show_id'], )
df['rating'] = df['rating'].apply(func=lambda x: x if x not in RATING_MAP.keys() else RATING_MAP[x])
df.columns

Index(['type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [26]:
movies = df[df['type'] == 'Movie']
series = df[df['type'] == 'TV Show']
#to make sure there isn't any other type other than movies and tv shows
filtered_df = df.query("type != 'Movie' & type != 'TV Show'")
filtered_df.shape

(6132, 11, 2677, 11) (8809, 11)


(0, 11)